In [2]:
# =========================
# Установка библиотек
# =========================
!pip install --upgrade --quiet pandas requests openpyxl nest_asyncio

# =========================
# Подключение async к Colab
# =========================
import nest_asyncio
nest_asyncio.apply()

# =========================
# Импорты
# =========================
import os
import json, uuid
import pandas as pd
import requests
import telebot
from telebot import types
import re
import traceback
from datetime import datetime

# =========================
# Настройки
# =========================
SBER_AUTH_KEY = "MDE5YjljNDQtMDQxMi03MWQ5LWIyYjMtYmI4YmIwZDViYzFhOjQ1YjcxMjY5LTExYjgtNDkyYy1iMzI1LWE5ZTU4MDQzNWE2Yw=="
TELEGRAM_TOKEN = "8175596339:AAFVwmGr0ZMbqNPyoq1xKiD6EnWX0ZIq-P8"
EXCEL_FILENAME = "LLM_Colab_Report.xlsx"

# =========================
# АДМИНИСТРАТОРЫ (по юзернейму)
# =========================
# Добавьте сюда юзернеймы администраторов (без @)
ADMIN_USERNAMES = {
    "padipicz",         # Ваш юзернейм @Padipicz
    "admin",            # Дополнительный администратор @admin
    "manager"           # Дополнительный администратор @manager
}

# =========================
# История диалогов
# =========================
conversation_history = {}

# =========================
# Функция проверки администратора по username
# =========================
def is_admin(user):
    """Проверяет, является ли пользователь администратором по username"""
    if not user:
        return False

    username = user.username
    if not username:
        # Если у пользователя нет username, можно также проверить по ID
        print(f"⚠️ У пользователя {user.id} нет username")
        return False

    # Приводим к нижнему регистру и убираем @
    username_clean = username.lower().replace('@', '')

    # Проверяем в списке администраторов (также приводим к нижнему регистру)
    admin_usernames_clean = [u.lower().replace('@', '') for u in ADMIN_USERNAMES]

    is_admin_user = username_clean in admin_usernames_clean

    if is_admin_user:
        print(f"✅ Пользователь @{username} - администратор")
    else:
        print(f"❌ Пользователь @{username} - не администратор")

    return is_admin_user

# =========================
# Функции для работы с GigaChat
# =========================
def get_token(auth_token: str) -> str:
    AUTH_URL = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"
    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "Accept": "application/json",
        "RqUID": str(uuid.uuid4()),
        "Authorization": f"Basic {auth_token}"
    }
    payload = {"scope": "GIGACHAT_API_PERS"}

    import urllib3
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

    response = requests.post(AUTH_URL, headers=headers, data=payload, verify=False)
    response.raise_for_status()
    return response.json()["access_token"]

def get_chat_completion(access_token: str, messages: list):
    GIGACHAT_CHAT_URL = "https://gigachat.devices.sberbank.ru/api/v1/chat/completions"
    payload = {
        "model": "GigaChat",
        "messages": messages,
        "temperature": 0.7,
        "top_p": 0.9,
        "n": 1,
        "stream": False,
        "max_tokens": 1000,
        "repetition_penalty": 1.0,
        "update_interval": 0
    }
    headers = {
        "Content-Type": "application/json",
        "Accept": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.post(GIGACHAT_CHAT_URL, headers=headers, data=json.dumps(payload), verify=False)
    response.raise_for_status()
    return response.json()["choices"][0]["message"]["content"]

# =========================
# Функция для парсинга "грязного" JSON
# =========================
def parse_dirty_json(dirty_text: str):
    """Парсит JSON с разными кавычками и форматированием"""

    # Очищаем текст
    text = dirty_text.strip()

    # Убираем обратные кавычки и код вокруг
    if '```' in text:
        parts = text.split('```')
        for part in parts:
            if '{' in part and '}' in part:
                text = part
                break

    # Убираем метки формата
    text = re.sub(r'JSONформат<.*?>', '', text, flags=re.IGNORECASE | re.DOTALL)
    text = re.sub(r'```(?:json)?', '', text)

    # Нормализуем кавычки
    text = text.replace('«', '"').replace('»', '"')
    text = text.replace('„', '"').replace('"', '"')
    text = text.replace('"', '"').replace('"', '"')
    text = text.replace('"', '"').replace('"', '"')
    text = text.replace('`', '"').replace("'", '"')

    # Исправляем названия полей
    field_corrections = {
        '"Intent»': '"intent"', '"intent»': '"intent"',
        '"Product': '"product"',
        '"Interest Level': '"interest_level"', '"Interest Level"': '"interest_level"',
        '"Urgenc y': '"urgency"', '"Urgency': '"urgency"',
        '"Summary "': '"summary"', '"Summary"': '"summary"',
        '"Suggeste д Reply': '"suggested_reply"', '"Suggested Reply': '"suggested_reply"',
        '"suggested_reply': '"suggested_reply"',
        '"urgency': '"urgency"',
        '"interest_level"': '"interest_level"',
    }

    for wrong, correct in field_corrections.items():
        text = text.replace(wrong, correct)

    # Убираем лишние пробелы и переносы
    text = re.sub(r':\s+', ': ', text)
    text = re.sub(r',\s+', ', ', text)

    # Находим JSON в тексте
    json_pattern = r'\{[^{}]*\}'
    matches = re.findall(json_pattern, text, re.DOTALL)

    for match in matches:
        try:
            return json.loads(match)
        except json.JSONDecodeError:
            continue

    # Если не нашли валидный JSON, пытаемся создать из полей
    result = {}

    patterns = {
        'intent': r'(?:intent|Intent)[\s:]+["\']([^"\']+)["\']',
        'product': r'(?:product|Product)[\s:]+["\']([^"\']+)["\']',
        'interest_level': r'(?:interest_level|Interest Level)[\s:]+["\']([^"\']+)["\']',
        'urgency': r'(?:urgency|Urgency)[\s:]+["\']([^"\']+)["\']',
        'summary': r'(?:summary|Summary)[\s:]+["\']([^"\']+)["\']',
        'suggested_reply': r'(?:suggested_reply|Suggested Reply)[\s:]+["\']([^"\']+)["\']'
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            result[key] = match.group(1).strip()

    return result

# =========================
# Обработка сообщений
# =========================
def process_message(session_id: str, user_message: str) -> str:
    """Обрабатывает сообщение, сохраняет в таблицу, возвращает только suggested_reply"""

    # Загружаем или создаем таблицу
    if os.path.exists(EXCEL_FILENAME):
        try:
            report_df = pd.read_excel(EXCEL_FILENAME)
        except:
            report_df = pd.DataFrame(columns=[
                "session_id", "user_message", "intent", "product",
                "interest_level", "urgency", "summary", "suggested_reply", "timestamp"
            ])
    else:
        report_df = pd.DataFrame(columns=[
            "session_id", "user_message", "intent", "product",
            "interest_level", "urgency", "summary", "suggested_reply", "timestamp"
        ])

    # Инициализация истории
    if session_id not in conversation_history:
        system_message = {
            "role": "system",
            "content": """Ты AI-ассистент для анализа сообщений клиентов банка.
Анализируй сообщения клиентов и возвращай результат в формате JSON со следующими полями:
1. intent - намерение клиента (например: "оформление кредита", "открытие вклада", "консультация")
2. product - продукт/услуга банка (например: "кредитная карта", "ипотека", "депозит")
3. interest_level - уровень интереса ("низкий", "средний", "высокий")
4. urgency - срочность ("низкая", "средняя", "высокая")
5. summary - краткое резюме сообщения
6. suggested_reply - рекомендация для консультанта как ответить клиенту

ВАЖНО: Отвечай ТОЛЬКО в формате JSON, без дополнительного текста."""
        }
        conversation_history[session_id] = [system_message]

    conversation_history[session_id].append({"role": "user", "content": user_message})

    # Ограничиваем историю
    if len(conversation_history[session_id]) > 8:
        conversation_history[session_id] = [conversation_history[session_id][0]] + conversation_history[session_id][-7:]

    try:
        # Получаем ответ от GigaChat
        access_token = get_token(SBER_AUTH_KEY)
        giga_response = get_chat_completion(access_token, conversation_history[session_id])
        conversation_history[session_id].append({"role": "assistant", "content": giga_response})

        print(f"📄 Ответ от GigaChat: {giga_response[:300]}...")

        # Парсим "грязный" JSON
        result_json = parse_dirty_json(giga_response)

        # Заполняем недостающие поля значениями по умолчанию
        default_values = {
            "intent": "не определено",
            "product": "не определено",
            "interest_level": "средний",
            "urgency": "средняя",
            "summary": user_message[:150] + ("..." if len(user_message) > 150 else ""),
            "suggested_reply": "Уточните детали запроса у клиента"
        }

        for key, default_value in default_values.items():
            if key not in result_json or not result_json[key]:
                result_json[key] = default_value

        # Получаем suggested_reply для ответа в Telegram
        telegram_reply = result_json.get("suggested_reply", "Уточните детали запроса у клиента")

        # Создаем запись для таблицы
        new_row = {
            "session_id": session_id,
            "user_message": user_message[:500],
            "intent": str(result_json.get("intent", ""))[:200],
            "product": str(result_json.get("product", ""))[:200],
            "interest_level": str(result_json.get("interest_level", ""))[:50],
            "urgency": str(result_json.get("urgency", ""))[:50],
            "summary": str(result_json.get("summary", ""))[:500],
            "suggested_reply": str(telegram_reply)[:500],
            "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        }

        print(f"📊 Извлеченные данные для таблицы:")
        for key, value in new_row.items():
            if key != 'session_id' and key != 'timestamp':
                print(f"  {key}: {value[:100]}")

        # Добавляем в таблицу
        new_df = pd.DataFrame([new_row])
        report_df = pd.concat([report_df, new_df], ignore_index=True)

        # Сохраняем Excel
        try:
            report_df.to_excel(EXCEL_FILENAME, index=False)
            print(f"✅ Данные сохранены в {EXCEL_FILENAME}. Всего записей: {len(report_df)}")
        except Exception as e:
            print(f"⚠️ Ошибка при сохранении Excel: {e}")
            try:
                with pd.ExcelWriter(EXCEL_FILENAME, engine='openpyxl') as writer:
                    report_df.to_excel(writer, index=False)
                print(f"✅ Данные сохранены через ExcelWriter")
            except Exception as e2:
                print(f"❌ Критическая ошибка сохранения: {e2}")

        # Возвращаем ТОЛЬКО suggested_reply для Telegram
        return telegram_reply

    except requests.exceptions.RequestException as e:
        print(f"❌ Ошибка подключения к API: {e}")
        return "⚠️ Ошибка подключения к AI-сервису. Попробуйте позже."
    except Exception as e:
        print(f"❌ Общая ошибка обработки: {traceback.format_exc()}")
        return "⚠️ Произошла ошибка при обработке сообщения. Попробуйте еще раз."

# =========================
# Создание бота
# =========================
bot = telebot.TeleBot(TELEGRAM_TOKEN)

# =========================
# ОБЫЧНЫЕ КОМАНДЫ (для всех пользователей)
# =========================

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
    welcome_text = """🤖 *Аналитический бот для клиентских сообщений*

*Как работает:*
1. 📝 Отправьте мне сообщение клиента
2. 🧠 Я проанализирую его через AI
3. 💡 Вы получите рекомендацию для ответа
4. 💾 Все данные сохранятся в Excel таблицу

*Доступные команды:*
/start - это сообщение
/report - статистика анализа
/myinfo - информация о вашем аккаунте

*Примеры сообщений для анализа:*
• "Хочу оформить кредитную карту"
• "Меня интересует ипотека"
• "Как открыть вклад с высоким процентом?"
"""
    bot.reply_to(message, welcome_text, parse_mode='Markdown')

@bot.message_handler(commands=['myinfo'])
def show_user_info(message):
    """Показывает информацию о пользователе"""
    user = message.from_user
    user_info = f"""👤 *Информация о вашем аккаунте:*

*ID:* `{user.id}`
*Имя:* {user.first_name or "Не указано"}
*Фамилия:* {user.last_name or "Не указано"}
*Юзернейм:* @{user.username or "Не установлен"}
*Язык:* {user.language_code or "Не определен"}

*Для администраторов:*
Если вы администратор, используйте /admin
"""
    bot.reply_to(message, user_info, parse_mode='Markdown')

@bot.message_handler(commands=['report'])
def send_report(message):
    """Статистика для всех пользователей"""
    try:
        if os.path.exists(EXCEL_FILENAME):
            report_df = pd.read_excel(EXCEL_FILENAME)
            total_messages = len(report_df)

            if total_messages > 0:
                # Базовая статистика (без деталей)
                report_text = f"""📊 *Общая статистика:*

• Сообщений проанализировано: {total_messages}
• Система работает исправно ✅

*Используйте бота для анализа сообщений клиентов!*
"""
            else:
                report_text = "📭 Пока не проанализировано ни одного сообщения."

            bot.reply_to(message, report_text, parse_mode='Markdown')
        else:
            bot.reply_to(message, "📭 Файл отчета еще не создан.")
    except Exception as e:
        bot.reply_to(message, f"❌ Ошибка: {str(e)[:100]}")

@bot.message_handler(commands=['clear'])
def clear_personal_history(message):
    """Очистка персональной истории (для всех пользователей)"""
    session_id = str(message.from_user.id)
    if session_id in conversation_history:
        if len(conversation_history[session_id]) > 1:
            conversation_history[session_id] = [conversation_history[session_id][0]]
            bot.reply_to(message, "✅ Ваша история диалога очищена!")
        else:
            bot.reply_to(message, "ℹ️ Ваша история уже пустая.")
    else:
        bot.reply_to(message, "ℹ️ У вас еще нет истории диалога.")

# =========================
# АДМИН КОМАНДЫ (только для администраторов по username)
# =========================

@bot.message_handler(commands=['admin'])
def admin_panel(message):
    """Панель администратора (только для админов по username)"""
    user = message.from_user

    print(f"🔍 Проверка админ-доступа для @{user.username}")

    if not is_admin(user):
        bot.reply_to(message, "⛔ *У вас нет прав доступа к админ-панели.*\n\n"
                              "Админ-панель доступна только пользователям с правами администратора.",
                    parse_mode='Markdown')
        return

    # Создаем клавиатуру админ-панели
    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, row_width=2)

    btn1 = types.KeyboardButton('📊 Полная статистика')
    btn2 = types.KeyboardButton('📥 Скачать таблицу')
    btn3 = types.KeyboardButton('🧹 Очистить всю историю')
    btn4 = types.KeyboardButton('👥 Список пользователей')
    btn5 = types.KeyboardButton('📋 Просмотреть таблицу')
    btn6 = types.KeyboardButton('❌ Закрыть админ-панель')

    markup.add(btn1, btn2, btn3, btn4, btn5, btn6)

    admin_text = f"""🔐 *АДМИН-ПАНЕЛЬ*

*Добро пожаловать, @{user.username}!*

Выберите действие:

*📊 Полная статистика* - детальная аналитика
*📥 Скачать таблицу* - получить Excel файл
*🧹 Очистить всю историю* - удалить ВСЕ данные
*👥 Список пользователей* - просмотр активных пользователей
*📋 Просмотреть таблицу* - просмотр данных без скачивания
*❌ Закрыть админ-панель* - вернуться в обычный режим

*Администраторы системы:*
{', '.join([f'@{username}' for username in ADMIN_USERNAMES])}
"""

    bot.send_message(message.chat.id, admin_text, parse_mode='Markdown', reply_markup=markup)

@bot.message_handler(func=lambda message: message.text == '📊 Полная статистика')
def admin_full_statistics(message):
    """Полная статистика для администратора"""
    user = message.from_user
    if not is_admin(user):
        bot.reply_to(message, "⛔ У вас нет прав доступа.")
        return

    try:
        if os.path.exists(EXCEL_FILENAME):
            report_df = pd.read_excel(EXCEL_FILENAME)
            total_messages = len(report_df)

            if total_messages > 0:
                # Детальная статистика только для админов
                unique_sessions = report_df['session_id'].nunique()

                # Самые частые интенты
                if 'intent' in report_df.columns and not report_df['intent'].empty:
                    intent_counts = report_df['intent'].value_counts()
                    top_intent = intent_counts.index[0] if len(intent_counts) > 0 else "нет данных"
                    top_intent_count = intent_counts.iloc[0] if len(intent_counts) > 0 else 0
                    top_intents = "\n".join([f"• {intent[:30]}: {count}" for intent, count in intent_counts.head(5).items()])
                else:
                    top_intent = "нет данных"
                    top_intent_count = 0
                    top_intents = "нет данных"

                # Распределение по продуктам
                if 'product' in report_df.columns:
                    product_counts = report_df['product'].value_counts()
                    top_products = "\n".join([f"• {product[:30]}: {count}" for product, count in product_counts.head(5).items()])
                else:
                    top_products = "нет данных"

                # Распределение по уровню интереса
                if 'interest_level' in report_df.columns:
                    interest_dist = report_df['interest_level'].value_counts()
                    interest_stats = "\n".join([f"• {level}: {count}" for level, count in interest_dist.items()])
                else:
                    interest_stats = "нет данных"

                # Временные метрики
                if 'timestamp' in report_df.columns:
                    report_df['timestamp'] = pd.to_datetime(report_df['timestamp'], errors='coerce')
                    last_message = report_df['timestamp'].max()
                    first_message = report_df['timestamp'].min()
                    time_range = f"с {first_message.strftime('%d.%m.%Y')} по {last_message.strftime('%d.%m.%Y')}"
                else:
                    time_range = "нет данных"

                report_text = f"""🔐 *ПОЛНАЯ СТАТИСТИКА ДЛЯ АДМИНИСТРАТОРА*

📈 *ОБЩАЯ ИНФОРМАЦИЯ:*
• Всего сообщений: {total_messages}
• Уникальных пользователей: {unique_sessions}
• Период: {time_range}

🎯 *ТОП НАМЕРЕНИЙ:*
{top_intents}

🏦 *ТОП ПРОДУКТОВ:*
{top_products}

📊 *РАСПРЕДЕЛЕНИЕ ПО ИНТЕРЕСУ:*
{interest_stats}

💾 *ФАЙЛ ОТЧЕТА:*
• Название: `{EXCEL_FILENAME}`
• Размер: {os.path.getsize(EXCEL_FILENAME) / 1024:.1f} KB
"""
            else:
                report_text = "📭 В отчете пока нет данных."

            bot.reply_to(message, report_text, parse_mode='Markdown')
        else:
            bot.reply_to(message, "📭 Файл отчета еще не создан.")
    except Exception as e:
        bot.reply_to(message, f"❌ Ошибка: {str(e)[:100]}")

@bot.message_handler(func=lambda message: message.text == '📥 Скачать таблицу')
def admin_download_table(message):
    """Скачивание таблицы администратором"""
    user = message.from_user
    if not is_admin(user):
        bot.reply_to(message, "⛔ У вас нет прав доступа.")
        return

    try:
        if os.path.exists(EXCEL_FILENAME):
            # Отправляем файл
            with open(EXCEL_FILENAME, 'rb') as file:
                bot.send_document(
                    message.chat.id,
                    file,
                    caption=f"🔐 *Администраторский доступ*\n👤 Администратор: @{user.username}\n📊 Excel таблица с полными данными",
                    visible_file_name="LLM_Full_Analysis_Report.xlsx"
                )

            # Информация о таблице
            report_df = pd.read_excel(EXCEL_FILENAME)
            total = len(report_df)
            last_update = report_df['timestamp'].max() if 'timestamp' in report_df.columns else "неизвестно"

            info_text = f"""✅ *Таблица отправлена администратору @{user.username}!*

📈 *Статистика:*
• Всего записей: {total}
• Последнее обновление: {last_update}
• Файл содержит все данные анализа
"""
            bot.reply_to(message, info_text, parse_mode='Markdown')
        else:
            bot.reply_to(message, "📭 Файл отчета еще не создан.")
    except Exception as e:
        bot.reply_to(message, f"❌ Ошибка при отправке файла: {str(e)[:100]}")

@bot.message_handler(func=lambda message: message.text == '📋 Просмотреть таблицу')
def admin_view_table(message):
    """Просмотр таблицы без скачивания"""
    user = message.from_user
    if not is_admin(user):
        bot.reply_to(message, "⛔ У вас нет прав доступа.")
        return

    try:
        if os.path.exists(EXCEL_FILENAME):
            report_df = pd.read_excel(EXCEL_FILENAME)
            total = len(report_df)

            if total > 0:
                # Показываем последние 10 записей
                last_records = report_df.tail(10)

                table_preview = f"🔐 *ПОСЛЕДНИЕ 10 ЗАПИСЕЙ (всего: {total}):*\n\n"

                for idx, row in last_records.iterrows():
                    table_preview += f"*Запись #{idx+1}:*\n"
                    table_preview += f"👤 User ID: `{row.get('session_id', 'N/A')}`\n"
                    table_preview += f"📝 Сообщение: {row.get('user_message', 'N/A')[:50]}...\n"
                    table_preview += f"🎯 Намерение: {row.get('intent', 'N/A')}\n"
                    table_preview += f"🏦 Продукт: {row.get('product', 'N/A')}\n"
                    table_preview += f"📅 Время: {row.get('timestamp', 'N/A')}\n"
                    table_preview += "─" * 30 + "\n"

                # Если таблица большая, отправляем частями
                if len(table_preview) > 4000:
                    parts = [table_preview[i:i+4000] for i in range(0, len(table_preview), 4000)]
                    for part in parts:
                        bot.send_message(message.chat.id, part, parse_mode='Markdown')
                else:
                    bot.send_message(message.chat.id, table_preview, parse_mode='Markdown')
            else:
                bot.reply_to(message, "📭 Таблица пустая.")
        else:
            bot.reply_to(message, "📭 Файл отчета еще не создан.")
    except Exception as e:
        bot.reply_to(message, f"❌ Ошибка при просмотре таблицы: {str(e)[:100]}")

@bot.message_handler(func=lambda message: message.text == '🧹 Очистить всю историю')
def admin_clear_all_history(message):
    """Очистка всей истории (только для админов)"""
    user = message.from_user
    if not is_admin(user):
        bot.reply_to(message, "⛔ У вас нет прав доступа.")
        return

    # Создаем клавиатуру подтверждения
    markup = types.InlineKeyboardMarkup()
    btn_yes = types.InlineKeyboardButton('✅ Да, очистить всё', callback_data='clear_all_yes')
    btn_no = types.InlineKeyboardButton('❌ Нет, отменить', callback_data='clear_all_no')
    markup.add(btn_yes, btn_no)

    warning_text = f"""⚠️ *ВНИМАНИЕ АДМИНИСТРАТОРА @{user.username}!*

🗑️ *Вы собираетесь очистить ВСЮ историю!*

*Будут удалены:*
• Вся таблица с данными анализа
• Вся история диалогов всех пользователей
• Все сохраненные записи

*ЭТО ДЕЙСТВИЕ НЕЛЬЗЯ ОТМЕНИТЬ!*

Вы уверены что хотите продолжить?"""

    bot.send_message(message.chat.id, warning_text, parse_mode='Markdown', reply_markup=markup)

@bot.callback_query_handler(func=lambda call: call.data.startswith('clear_all_'))
def handle_clear_confirmation(call):
    """Обработка подтверждения очистки"""
    user = call.from_user

    # Проверяем права администратора
    if not is_admin(user):
        bot.answer_callback_query(call.id, "⛔ У вас нет прав доступа.")
        return

    if call.data == 'clear_all_yes':
        try:
            # Очищаем таблицу
            if os.path.exists(EXCEL_FILENAME):
                os.remove(EXCEL_FILENAME)

            # Очищаем историю диалогов
            global conversation_history
            conversation_history = {}

            bot.edit_message_text(
                chat_id=call.message.chat.id,
                message_id=call.message.message_id,
                text=f"✅ *ВСЯ ИСТОРИЯ ОЧИЩЕНА АДМИНИСТРАТОРОМ @{user.username}!*\n\n🗑️ Удалены:\n• Таблица с данными\n• История диалогов\n• Все записи",
                parse_mode='Markdown'
            )

            print(f"⚠️ Администратор @{user.username} очистил всю историю!")

        except Exception as e:
            bot.edit_message_text(
                chat_id=call.message.chat.id,
                message_id=call.message.message_id,
                text=f"❌ Ошибка при очистке: {str(e)[:100]}"
            )

    elif call.data == 'clear_all_no':
        bot.edit_message_text(
            chat_id=call.message.chat.id,
            message_id=call.message.message_id,
            text="✅ *Очистка отменена*\n\nДанные сохранены."
        )

@bot.message_handler(func=lambda message: message.text == '👥 Список пользователей')
def admin_user_list(message):
    """Список активных пользователей"""
    user = message.from_user
    if not is_admin(user):
        bot.reply_to(message, "⛔ У вас нет прав доступа.")
        return

    try:
        if os.path.exists(EXCEL_FILENAME):
            report_df = pd.read_excel(EXCEL_FILENAME)

            if len(report_df) > 0:
                # Группируем по пользователям
                user_stats = report_df.groupby('session_id').agg({
                    'user_message': 'count',
                    'timestamp': ['min', 'max']
                }).reset_index()

                user_stats.columns = ['user_id', 'message_count', 'first_message', 'last_message']

                user_list_text = f"🔐 *СПИСОК АКТИВНЫХ ПОЛЬЗОВАТЕЛЕЙ:*\n\n"

                for idx, row in user_stats.iterrows():
                    user_list_text += f"*👤 Пользователь #{idx+1}:*\n"
                    user_list_text += f"• ID пользователя: `{row['user_id']}`\n"
                    user_list_text += f"• Сообщений: {row['message_count']}\n"
                    user_list_text += f"• Первое сообщение: {row['first_message']}\n"
                    user_list_text += f"• Последнее сообщение: {row['last_message']}\n"
                    user_list_text += "─" * 30 + "\n"

                user_list_text += f"\n*Всего уникальных пользователей:* {len(user_stats)}"

                bot.send_message(message.chat.id, user_list_text, parse_mode='Markdown')
            else:
                bot.reply_to(message, "📭 Нет данных о пользователях.")
        else:
            bot.reply_to(message, "📭 Файл отчета еще не создан.")
    except Exception as e:
        bot.reply_to(message, f"❌ Ошибка: {str(e)[:100]}")

@bot.message_handler(func=lambda message: message.text == '❌ Закрыть админ-панель')
def admin_close_panel(message):
    """Закрытие админ-панели"""
    user = message.from_user
    if not is_admin(user):
        bot.reply_to(message, "⛔ У вас нет прав доступа.")
        return

    # Убираем клавиатуру
    markup = types.ReplyKeyboardRemove()
    bot.send_message(
        message.chat.id,
        f"✅ *Админ-панель закрыта*\n\nВозврат в обычный режим работы, @{user.username}.",
        parse_mode='Markdown',
        reply_markup=markup
    )

# =========================
# Обработка обычных сообщений (для всех пользователей)
# =========================

@bot.message_handler(func=lambda message: True)
def handle_all_messages(message):
    # Пропускаем админ-команды из кнопок
    if message.text in ['📊 Полная статистика', '📥 Скачать таблицу', '🧹 Очистить всю историю',
                       '👥 Список пользователей', '📋 Просмотреть таблицу', '❌ Закрыть админ-панель']:
        return

    try:
        # Показываем статус "печатает"
        bot.send_chat_action(message.chat.id, 'typing')

        session_id = str(message.from_user.id)
        user_message = message.text

        print(f"📨 Входящее сообщение от {session_id} (@{message.from_user.username}): {user_message[:100]}...")

        # Обработка сообщения
        reply = process_message(session_id, user_message)

        # Отправляем ответ (только suggested_reply)
        response_text = f"""💡 *Рекомендация для консультанта:*

{reply}

📊 *Данные сохранены в таблицу*
📈 Статистика: /report
👤 Ваша информация: /myinfo
"""

        bot.reply_to(message, response_text, parse_mode='Markdown')
        print(f"✅ Ответ отправлен пользователю @{message.from_user.username}: {reply[:100]}...")

    except Exception as e:
        error_msg = f"❌ Ошибка обработки сообщения. Попробуйте еще раз.\n\nОшибка: {str(e)[:200]}"
        bot.reply_to(message, error_msg)
        print(f"❌ Критическая ошибка: {traceback.format_exc()}")

# =========================
# Запуск бота
# =========================
print("=" * 70)
print("🤖 АНАЛИТИЧЕСКИЙ БОТ С АДМИН-ПАНЕЛЬЮ ПО USERNAME")
print("=" * 70)
print(f"📁 Файл отчета: {EXCEL_FILENAME}")
print(f"👑 Администраторы: {', '.join([f'@{username}' for username in ADMIN_USERNAMES])}")
print("=" * 70)
print("📱 *Клиентские функции:*")
print("• Анализ сообщений через AI (/start)")
print("• Базовая статистика (/report)")
print("• Информация об аккаунте (/myinfo)")
print("• Очистка личной истории (/clear)")
print("=" * 70)
print("🔐 *Администраторские функции (только для указанных username):*")
print("• Полная аналитика (/admin)")
print("• Скачивание таблицы")
print("• Очистка всей истории")
print("• Просмотр пользователей")
print("=" * 70)
print("🚀 Запуск бота...")
print("📝 Используйте /start для начала работы")
print("👤 Проверьте свой аккаунт: /myinfo")
print("🔐 Администраторы: используйте /admin")
print("=" * 70)

try:
    bot.polling(none_stop=True, interval=2, timeout=60)
    print("✅ Бот успешно запущен и работает!")
except Exception as e:
    print(f"❌ Ошибка запуска бота: {e}")
    print("⚠️ Возможные причины:")
    print("   - Неверный токен Telegram")
    print("   - Проблемы с подключением к интернету")
    print("   - Ограничения Colab на внешние соединения")

🤖 АНАЛИТИЧЕСКИЙ БОТ С АДМИН-ПАНЕЛЬЮ ПО USERNAME
📁 Файл отчета: LLM_Colab_Report.xlsx
👑 Администраторы: @manager, @admin, @padipicz
📱 *Клиентские функции:*
• Анализ сообщений через AI (/start)
• Базовая статистика (/report)
• Информация об аккаунте (/myinfo)
• Очистка личной истории (/clear)
🔐 *Администраторские функции (только для указанных username):*
• Полная аналитика (/admin)
• Скачивание таблицы
• Очистка всей истории
• Просмотр пользователей
🚀 Запуск бота...
📝 Используйте /start для начала работы
👤 Проверьте свой аккаунт: /myinfo
🔐 Администраторы: используйте /admin
🔍 Проверка админ-доступа для @Padipicz
✅ Пользователь @Padipicz - администратор
✅ Пользователь @Padipicz - администратор
✅ Пользователь @Padipicz - администратор
✅ Пользователь @Padipicz - администратор
✅ Пользователь @Padipicz - администратор
✅ Пользователь @Padipicz - администратор
⚠️ Администратор @Padipicz очистил всю историю!
✅ Пользователь @Padipicz - администратор
✅ Пользователь @Padipicz - администратор
✅ Поль